In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# กำหนดค่า API URL, USERNAME, PASSWORD ของ Netpie
API_URL = "https://api.netpie.io/v2/device/shadow/data"
USERNAME = "Token Device"
PASSWORD = "Secret Device"

# ดึงข้อมูลจาก Netpie
response = requests.get(API_URL, auth=HTTPBasicAuth(USERNAME, PASSWORD))

# ตรวจสอบว่าข้อมูลดึงมาถูกต้องหรือไม่
if response.status_code == 200:
    # แปลงข้อมูลที่ได้รับเป็น JSON
    data = response.json()

    # พิมพ์ข้อมูลทั้งหมดที่ได้รับจาก API
    print("ข้อมูลที่ได้รับจาก Netpie:")
    print(data)

    # สร้างข้อความจากข้อมูลที่ได้รับทั้งหมด
    message = f"""
    🚨 แจ้งเตือนจาก NetPIE 🚨

    🌡️ **อุณหภูมิ**: {data['data'].get('temperature', 'ไม่ระบุ')} °C
    💧 **ความชื้น**: {data['data'].get('humidity', 'ไม่ระบุ')} %
    🌊 **ระดับน้ำ**: {data['data'].get('water_level', 'ไม่ระบุ')}
    🌱 **ความชื้นในดิน**: {data['data'].get('soil_moisture', 'ไม่ระบุ')}
    🌀 **ค่า MQ Sensor**: {data['data'].get('mq_sensor', 'ไม่ระบุ')}

    📅 **เวลาที่แจ้งเตือนข้อมูล**: {data.get('timestamp', 'ไม่ระบุ')}

    ขอบคุณที่ใช้บริการ NetPIE!
    """

    # ส่งข้อมูลไปยัง LINE
    line_api_url = "https://api.line.me/v2/bot/message/broadcast"
    headers = {
        'Authorization': 'Bearer ',  # ใส่ Access Token ของ LINE 
        'Content-Type': 'application/json'
    }

    payload = {
        "messages": [
            {
                "type": "text",
                "text": message
            }
        ]
    }

    # ส่ง POST Request ไปยัง LINE API
    line_response = requests.post(line_api_url, headers=headers, json=payload)

    # ตรวจสอบผลลัพธ์การส่งข้อความ
    if line_response.status_code == 200:
        print("ส่งข้อความไปยัง LINE สำเร็จ")
    else:
        print(f"เกิดข้อผิดพลาดในการส่งข้อความไปยัง LINE: {line_response.text}")
else:
    print(f"เกิดข้อผิดพลาดในการดึงข้อมูลจาก Netpie: {response.status_code}")


In [ ]:
import requests
import json

url = "https://api.netpie.io/v2/feed/api/v1/datapoints/query"

headers = {
    "Authorization": "Bearer ", # ใส่ Access Token ของ LINE 
    "Content-Type": "application/json"
}

payload = {
    "start_relative": {
        "value": 1,
        "unit": "days"
    },
    "metrics": [
        {
            "name": "Token Device",
            "tags": {
                "attr": ["temperature", "humidity", "water_level", "soil_moisture", "mq_sensor"]
            },
            "limit": 50,
            "group_by": [
                {
                    "name": "tag",
                    "tags": ["attr"]
                }
            ],
            "aggregators": [
                {
                    "name": "avg",
                    "sampling": {
                        "value": "1",
                        "unit": "minutes"
                    }
                }
            ]
        }
    ]
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    results = data.get("queries", [{}])[0].get("results", [])

    attr_data = {}

    for result in results:
        attr = result.get("tags", {}).get("attr", [None])[0]
        values = result.get("values", [])
        if attr and values:
            attr_data[attr] = [v[1] for v in values]  # ดึงแต่ค่า value

    # หาจำนวนข้อมูลชุดที่น้อยที่สุด เพื่อความปลอดภัยเวลา zip
    min_len = min(len(v) for v in attr_data.values())

    combined_data = []
    for i in range(min_len):
        entry = {
            "temperature": attr_data.get("temperature", [None]*min_len)[i],
            "humidity": attr_data.get("humidity", [None]*min_len)[i],
            "water_level": attr_data.get("water_level", [None]*min_len)[i],
            "soil_moisture": attr_data.get("soil_moisture", [None]*min_len)[i],
            "mq_sensor": attr_data.get("mq_sensor", [None]*min_len)[i],
        }
        combined_data.append(entry)

    # บันทึกเป็น JSON ไฟล์
    with open("/content/data.json", "w") as f:
        json.dump(combined_data, f, indent=2)

    print("✅ บันทึกไฟล์ JSON เรียบร้อยแล้วที่ /content/data.json")
else:
    print("❌ Error:", response.status_code, response.text)


In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from requests.auth import HTTPBasicAuth
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras
from datetime import datetime

# --- NETPIE Auth ---
USERNAME = "Token Device"
PASSWORD = "Secret Device"
API_URL = "https://api.netpie.io/v2/device/shadow/data"

# --- Load historical data for training ---
def load_training_data():
    try:
        with open("data.json", "r") as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        df["growth_rate"] = (df["temperature"] * 0.5) + (df["humidity"] * 0.3) - (df["water_level"] * 0.2) + (df["soil_moisture"] * 0.1)
        return df
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print("❌ Error loading training data:", e)
        return None

# --- Fetch data from NETPIE ---
def fetch_latest_data():
    response = requests.get(API_URL, auth=HTTPBasicAuth(USERNAME, PASSWORD))
    if response.status_code == 200:
        data = response.json()
        return data.get("data", None)
    else:
        print("❌ Error fetching data:", response.status_code)
        return None

# --- Save latest prediction to separate file ---
def save_latest_prediction(latest_record):
    try:
        with open("latest_prediction.json", "w") as f:
            json.dump(latest_record, f, indent=4)
        print("✅ Saved latest prediction to latest_prediction.json")
    except Exception as e:
        print("❌ Error saving latest prediction:", e)

# --- Main Workflow ---
df = load_training_data()
if df is not None:
    # Prepare training data
    X = df[["temperature", "humidity", "water_level", "soil_moisture"]]
    y = df["growth_rate"]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # --- Train Random Forest with GridSearch ---
    rf_model = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }
    grid_search_rf = GridSearchCV(rf_model, param_grid, cv=5, n_jobs=-1, verbose=1)
    grid_search_rf.fit(X_scaled, y)

    best_rf_model = grid_search_rf.best_estimator_
    print("Best Random Forest Parameters:", grid_search_rf.best_params_)

    # --- Train Neural Network ---
    nn_model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(X_scaled.shape[1],)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1)
    ])
    nn_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    nn_model.fit(X_scaled, y, epochs=200, batch_size=32, verbose=0)

    # Fetch latest data and predict
    latest_data = fetch_latest_data()
    if latest_data:
        try:
            input_data = np.array([[latest_data["temperature"], latest_data["humidity"],
                                    latest_data["water_level"], latest_data["soil_moisture"]]])

            # Predict
            rf_pred = best_rf_model.predict(scaler.transform(input_data))[0]
            nn_pred = nn_model.predict(scaler.transform(input_data))[0][0]

            print("\n📊 Real-time Prediction")
            print(f"🌱 Random Forest Growth Rate: {rf_pred:.2f}%")
            print(f"🌿 Neural Network Growth Rate: {nn_pred:.2f}%")

            # Convert float32 to float before saving
            latest_data["growth_rate_rf"] = round(float(rf_pred), 2)
            latest_data["growth_rate_nn"] = round(float(nn_pred), 2)
            latest_data["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Save the latest prediction
            save_latest_prediction(latest_data)

        except KeyError as e:
            print(f"⚠️ Missing key in latest data: {e}")
    else:
        print("⚠️ No data fetched from NETPIE.")
else:
    print("⚠️ Training data unavailable.")
